# Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader

# Main Code

In [ ]:
# Example features
categorical_cols = ['machine_type', 'glue_type', 'wood_type']  # Update as needed
numerical_cols = ['temperature', 'pressure', 'moisture', 'speed']  # Update as needed
target_cols = ['accuracy', 'tensile_strength', 'waste_material']

# Load your dataset here
# df = pd.read_csv('your_data.csv')

# Encode categorical columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Scale numerical columns
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Split data
X = df[categorical_cols + numerical_cols]
y = df[target_cols]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Torch Dataset
class TabularDataset(Dataset):
    def __init__(self, X, y, cat_cols, num_cols):
        self.cat_data = torch.tensor(X[cat_cols].values, dtype=torch.long)
        self.num_data = torch.tensor(X[num_cols].values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.cat_data[idx], self.num_data[idx], self.y[idx]

train_ds = TabularDataset(X_train, y_train, categorical_cols, numerical_cols)
val_ds = TabularDataset(X_val, y_val, categorical_cols, numerical_cols)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

# Neural Network
class TabularModel(nn.Module):
    def __init__(self, embedding_sizes, num_numerical, output_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(categories, min(50, (categories + 1) // 2))
            for categories, _ in embedding_sizes
        ])
        self.emb_drop = nn.Dropout(0.1)
        emb_dim = sum([min(50, (categories + 1) // 2) for categories, _ in embedding_sizes])

        self.fc = nn.Sequential(
            nn.Linear(emb_dim + num_numerical, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            nn.Linear(64, output_dim)
        )

    def forward(self, x_cat, x_num):
        x = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x = torch.cat([x, x_num], 1)
        return self.fc(x)

# Prepare embedding sizes
embedding_sizes = [(df[col].nunique(), 1) for col in categorical_cols]
model = TabularModel(embedding_sizes, num_numerical=len(numerical_cols), output_dim=len(target_cols))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training Loop

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_model(model, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for x_cat, x_num, y in train_loader:
            x_cat, x_num, y = x_cat.to(device), x_num.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x_cat, x_num)
            loss = loss_fn(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

train_model(model)

# Evaluation

In [ ]:
model.eval()
with torch.no_grad():
    y_true, y_pred = [], []
    for x_cat, x_num, y in val_loader:
        x_cat, x_num = x_cat.to(device), x_num.to(device)
        outputs = model(x_cat, x_num)
        y_true.append(y)
        y_pred.append(outputs.cpu())

    y_true = torch.cat(y_true, 0).numpy()
    y_pred = torch.cat(y_pred, 0).numpy()

from sklearn.metrics import mean_squared_error, r2_score

for i, col in enumerate(target_cols):
    rmse = mean_squared_error(y_true[:, i], y_pred[:, i], squared=False)
    r2 = r2_score(y_true[:, i], y_pred[:, i])
    print(f"{col.upper()} - RMSE: {rmse:.3f}, R2: {r2:.3f}")